# About

Another machine translation sketch, based on the [torchtext tutorial](https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html).

For comparison with the [seq2seq version](MT-pytorch-seq2seq-tutorial.ipynb), using the same Spanish-English dataset from [Anki](http://www.manythings.org/anki/).

## Preparing the data

In [8]:
!if [ ! -f ./data/spa-eng.zip ]; then wget -P ./data https://www.manythings.org/anki/spa-eng.zip; fi

In [9]:
!if [ ! -f ./data/spa.txt ]; then unzip -n -d ./data ./data/spa-eng.zip; fi

In [7]:
%%bash
pip install --upgrade pip
pip3 install spacy
python3 -m spacy download en
python3 -m spacy download es

  Found existing installation: pip 19.2.1
    Uninstalling pip-19.2.1:
      Successfully uninstalled pip-19.2.1
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047106 sha256=e23f2cab735ae62f3df76772e3e827cc9bcfc5cda8c072bd0ba2ac6b30a402dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-8gi74f4k/wheels/10/6f/a6/ddd8204ceecdedddea923f8514e13afb0c1f0f556d2c9c3da0
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.3.1-py3-none-any.whl size=16216999 sha256=5d3ae233cbef077a9419b9148da7ee220492a2cbc4f4eabbcc00d3045aaea331
  Stored in directory: /tmp/pip-ephem-wheel-cache-ed50iq10/wheels/ef/67/82/0b6f50c9f4a2ad0e01633af17ece1d0bd

The [torchtext.data](https://torchtext.readthedocs.io/en/latest/data.html) needs a parser, for which [spaCy](https://spacy.io/) is useful:

```sh
$ pip install spacy

$ python -m spacy download en
...
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/denis/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/home/denis/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')

$ python -m spacy download es
Collecting es_core_news_sm==2.2.5
...
Successfully built es-core-news-sm
Installing collected packages: es-core-news-sm
Successfully installed es-core-news-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/home/denis/anaconda3/lib/python3.7/site-packages/es_core_news_sm -->
/home/denis/anaconda3/lib/python3.7/site-packages/spacy/data/es
You can now load the model via spacy.load('es')
```

In [4]:
from torchtext.data import Field, BucketIterator, TabularDataset

In [10]:
'''
format of the samples is:
[english] tab [spanish]
''' 

SRC = Field(tokenize = "spacy",
            tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True,
            sequential=True)

TGT = Field(tokenize = "spacy",
            tokenizer_language="es",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True,
            sequential=True,
            is_target=True)

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [33]:
samples = TabularDataset(path='./data/spa.txt',
                         format='tsv', 
                         fields = [("src", SRC), ("tgt", TGT)],
                         skip_header=True)

(training, testing, validating) = samples.split(split_ratio=[0.6,0.2,0.2])

In [34]:
(len(training),len(testing),len(validating))

(75267, 25089, 25089)

In [35]:
SRC.build_vocab(training, min_freq = 2)
TGT.build_vocab(training, min_freq = 2)

In [36]:
len(SRC.vocab)

7281

In [37]:
len(TGT.vocab)

12015

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

import random
from typing import Tuple

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [38]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (training, testing, validating),
    batch_size = BATCH_SIZE,
    device = device,
    sort=False)

## Defining the model

In [17]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src: Tensor) -> Tuple[Tensor]:
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        return outputs, hidden

In [18]:
class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim
        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:
        src_len = encoder_outputs.shape[0]
        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))
        attention = torch.sum(energy, dim=2)
        return F.softmax(attention, dim=1)

In [19]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:
        a = self.attention(decoder_hidden, encoder_outputs)
        a = a.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted_encoder_rep = torch.bmm(a, encoder_outputs)
        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)
        return weighted_encoder_rep

    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)
        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)
        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)
        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))
        return output, decoder_hidden.squeeze(0)

In [20]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs

## Define an instance of the model with the data

In [39]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TGT.vocab)

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss(ignore_index=TGT.vocab.stoi['<pad>'])

def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,411,671 trainable parameters


## Training functions

In [22]:
import math
import time

def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):
    
    model.train()
    epoch_loss = 0

    for _, batch in enumerate(iterator):
        src = batch.src
        tgt = batch.tgt
        optimizer.zero_grad()
        output = model(src, tgt)
        output = output[1:].view(-1, output.shape[-1])
        tgt = tgt[1:].view(-1)
        loss = criterion(output, tgt)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):

    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for _, batch in enumerate(iterator):
            src = batch.src
            tgt = batch.tgt
            output = model(src, tgt, 0) #turn off teacher forcing
            output = output[1:].view(-1, output.shape[-1])
            tgt = tgt[1:].view(-1)
            loss = criterion(output, tgt)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

def epoch_time(start_time: int,
               end_time: int):
    
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Training attempt

In [40]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

    
test_loss = evaluate(model, test_iterator, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Epoch: 01 | Time: 1m 34s
	Train Loss: 5.579 | Train PPL: 264.787
	 Val. Loss: 5.251 |  Val. PPL: 190.815
Epoch: 02 | Time: 1m 36s
	Train Loss: 4.891 | Train PPL: 133.042
	 Val. Loss: 5.003 |  Val. PPL: 148.905
Epoch: 03 | Time: 1m 34s
	Train Loss: 4.630 | Train PPL: 102.564
	 Val. Loss: 4.827 |  Val. PPL: 124.793
Epoch: 04 | Time: 1m 34s
	Train Loss: 4.400 | Train PPL:  81.433
	 Val. Loss: 4.636 |  Val. PPL: 103.174
Epoch: 05 | Time: 1m 35s
	Train Loss: 4.174 | Train PPL:  64.951
	 Val. Loss: 4.409 |  Val. PPL:  82.158
Epoch: 06 | Time: 1m 35s
	Train Loss: 3.930 | Train PPL:  50.891
	 Val. Loss: 4.145 |  Val. PPL:  63.126
Epoch: 07 | Time: 1m 34s
	Train Loss: 3.667 | Train PPL:  39.143
	 Val. Loss: 3.953 |  Val. PPL:  52.089
Epoch: 08 | Time: 1m 34s
	Train Loss: 3.444 | Train PPL:  31.319
	 Val. Loss: 3.751 |  Val. PPL:  42.580
Epoch: 09 | Time: 1m 35s
	Train Loss: 3.245 | Train PPL:  25.660
	 Val. Loss: 3.581 |  Val. PPL:  35.915
Epoch: 10 | Time: 1m 35s
	Train Loss: 3.085 | Train PPL

## Inference

Use the trained model to come up with translations

In [66]:
def translate_sentence(sentence, src_field, tgt_field, model, device, max_len, src_lang):
    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load(src_lang)
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor) #, src_len)

    tgt_indexes = [tgt_field.vocab.stoi[tgt_field.init_token]]
    
    for i in range(max_len):
        tgt_tensor = torch.LongTensor([tgt_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden = model.decoder(tgt_tensor, hidden, encoder_outputs)

        pred_token = output.argmax(1).item()
        tgt_indexes.append(pred_token)

        if pred_token == tgt_field.vocab.stoi[tgt_field.eos_token]:
            break
    
    tgt_tokens = [tgt_field.vocab.itos[i] for i in tgt_indexes] 
    return tgt_tokens[1:]

In [69]:
'''
How to inspect the TabularDataset,
and request a translation
'''

example_item = training.examples[12] # 12 as a random index

src = vars(example_item)['src']
tgt = vars(example_item)['tgt']

print(f'src = {src}')
print(f'tgt = {tgt}')

# use the src value as the input
translation = translate_sentence(src, SRC, TGT, model, device, 100, 'en')

print(f'predicted = {translation}')

src = ['i', 'will', 'give', 'you', 'a', 'bicycle', 'for', 'your', 'birthday', '.']
tgt = ['te', 'regalaré', 'una', 'bicicleta', 'para', 'tu', 'cumpleaños', '.']
predicted = ['te', 'gustaría', 'un', 'coche', 'tu', 'tu', '.', '<eos>']


In [71]:
def evaluate_item(item):
    src = vars(item)['src']
    tgt = vars(item)['tgt']
    print('>', f'{src}')
    print('=', f'{tgt}')
    translation = translate_sentence(src, SRC, TGT, model, device, 100, 'en')
    print('<', f'{translation}')
    print('')

In [73]:
# sample some random entries

for i in range(30):
    evaluate_item(random.choice(samples))

> ['your', 'sister', 'is', 'a', 'good', 'pianist', ',', 'is', "n't", 'she', '?']
= ['tu', 'hermana', 'es', 'una', 'buena', 'pianista', ',', '¿', 'verdad', '?']
< ['tu', 'hijo', 'es', 'un', 'buen', 'hombre', ',', '¿', 'no', 'es', '?', '<eos>']

> ['he', 'carried', 'a', 'bundle', 'of', 'clothes', '.']
= ['llevaba', 'un', 'lío', 'de', 'ropa', '.']
< ['él', 'se', 'un', 'un', 'de', 'de', '.', '<eos>']

> ['i', 'do', "n't", 'think', 'people', 'should', 'make', 'a', 'mountain', 'of', 'a', 'mole', 'hill', '.']
= ['no', 'creo', 'que', 'la', 'gente', 'deba', 'hacer', 'una', 'montaña', 'de', 'un', 'grano', 'de', 'arena', '.']
< ['no', 'me', 'gusta', 'la', 'gente', 'que', '<unk>', 'un', 'accidente', 'de', 'un', '<unk>', '.', '<eos>']

> ['i', 'thought', 'it', 'was', 'a', 'mistake', 'to', 'tell', 'tom', 'the', 'secret', '.']
= ['pensé', 'que', 'era', 'un', 'error', 'decirle', 'a', 'tom', 'el', 'secreto', '.']
< ['pensé', 'que', 'era', 'un', 'un', 'que', 'tom', 'que', 'la', 'la', '.', '<eos>']

> ['